# Hands-On Pertemuan 6: Data Processing dengan Apache Spark

## Tujuan:
- Memahami dan mempraktikkan data processing menggunakan Apache Spark.
- Menggunakan Spark untuk operasi data yang efisien pada dataset besar.
- Menerapkan teknik canggih dalam Spark untuk mengatasi kasus penggunaan nyata.

### 1. Pengenalan Spark DataFrames
Spark DataFrame menyediakan struktur data yang optimal dengan operasi yang dioptimalkan untuk pemrosesan data besar, yang sangat mirip dengan DataFrame di Pandas atau di RDBMS.

- **Tugas 1**: Buat DataFrame sederhana di Spark dan eksplorasi beberapa fungsi dasar yang tersedia.

In [ ]:
!pip install pandas==2.2.2 numpy pyspark==3.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285391 sha256=9794cf4712aca45b94a6620794edaecc3c4437c5c9676053c7f52ac71e922bb0
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.3
    Uninstalling pyspark-3.5.3:
      Successfully uninstalled pyspark-3.5.3


In [ ]:
# Contoh membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan6').getOrCreate()

data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       James|     Sales|  3000|
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|       Maria|   Finance|  3000|
+------------+----------+------+



### 2. Transformasi Dasar dengan DataFrames


In [ ]:
df.select('EmployeeName', 'Salary').show()
df.filter(df['Salary'] > 3000).show()
df.groupBy('Department').avg('Salary').show()
df.groupBy('Department').sum('Salary').show()
df.agg({'Salary': 'avg'}).show()
df.agg({'Salary': 'max'}).show()

+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
+------------+----------+------+

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3900.0|
|   Finance|     3000.0|
+----------+-----------+

+----------+-----------+
|Department|sum(Salary)|
+----------+-----------+
|     Sales|      11700|
|   Finance|       3000|
+----------+-----------+

+-----------+
|avg(Salary)|
+-----------+
|     3675.0|
+-----------+

+-----------+
|max(Salary)|
+-----------+
|       4600|
+-----------+



### 3. Bekerja dengan Tipe Data Kompleks


In [ ]:
df_bonus = df.withColumn('SalaryBonus', df['Salary'] * 0.1)
df_total = df_bonus.withColumn('TotalCompensation', df_bonus['Salary'] + df_bonus['SalaryBonus'])
df_total.show()

+------------+----------+------+-----------+-----------------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|
+------------+----------+------+-----------+-----------------+
|       James|     Sales|  3000|      300.0|           3300.0|
|     Michael|     Sales|  4600|      460.0|           5060.0|
|      Robert|     Sales|  4100|      410.0|           4510.0|
|       Maria|   Finance|  3000|      300.0|           3300.0|
+------------+----------+------+-----------+-----------------+



### 4. Operasi Data Lanjutan


In [ ]:
# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df_with_rank = df.withColumn('Rank', F.rank().over(windowSpec))
df_with_running_total = df_with_rank.withColumn('RunningTotal', F.sum('Salary').over(windowSpec))
df_with_running_total.select('EmployeeName', 'Department', 'Salary', 'Rank', 'RunningTotal').show()

+------------+----------+------+----+------------+
|EmployeeName|Department|Salary|Rank|RunningTotal|
+------------+----------+------+----+------------+
|       Maria|   Finance|  3000|   1|        3000|
|       James|     Sales|  3000|   1|        3000|
|      Robert|     Sales|  4100|   2|        7100|
|     Michael|     Sales|  4600|   3|       11700|
+------------+----------+------+----+------------+

